<a href="https://colab.research.google.com/github/antariksh2648/Pair-Trading-Implementation/blob/main/Pair_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the libraries
!pip install pandas_datareader
import numpy as np
import pandas as pd
import statsmodels
from statsmodels.tsa.stattools import coint   # use this library to find this out the p_value
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import datetime

In [2]:
def downloadStockData(tickers,start,end):
    """
        This function downloads the Close price of the stock and creates a dataframe from it
        Params:
            tickers : a list of ticker of various stocks
            start   : it is the start date of the data to be downloaded. It is a datetime in datetime format.
            end     : it is the end date of the data to be downloaded. It is a datetime in datetime format.

        Return:
            A dataframe with all the close for of each stock on each open market day
    """
    ticker_data = pdr.get_data_yahoo(tickers, start, end)['Adj Close']
    return ticker_data

In [10]:
!pip install yfinance
import yfinance as yf
tickers = ['ADANIPORTS.NS',
 'TECHM.NS',
 'BRITANNIA.NS',
 'TCS.NS',
 'SUNPHARMA.NS',
 'INFY.NS',
 'HINDALCO.NS',
 'DRREDDY.NS',
 'HCLTECH.NS',
 'HINDUNILVR.NS',
 'TATASTEEL.NS',
 'NTPC.NS',
 'CIPLA.NS',
 'ITC.NS',
 'BPCL.NS',
 'WIPRO.NS',
 'JSWSTEEL.NS',
 'GAIL.NS',
 'DIVISLAB.NS',
 'ULTRACEMCO.NS',
 'NESTLEIND.NS',
 'ASIANPAINT.NS',
 'EICHERMOT.NS',
 'IOC.NS',
 'GRASIM.NS',
 'BAJAJ-AUTO.NS',
 'HDFC.NS',
 'M&M.NS',
 'UPL.NS',
 'SHREECEM.NS',
 'MARUTI.NS',
 'TITAN.NS',
 'BAJAJFINSV.NS',
 'KOTAKBANK.NS',
 'COALINDIA.NS',
 'BHARTIARTL.NS',
 'HEROMOTOCO.NS',
 'LT.NS',
 'ONGC.NS',
 'SBIN.NS',
 'RELIANCE.NS',
 'BAJFINANCE.NS',
 'AXISBANK.NS',
 'HDFCBANK.NS',
 'TATAMOTORS.NS',
 'ICICIBANK.NS',
 'POWERGRID.NS',
 'INDUSINDBK.NS']
start = datetime.datetime(2013,1,1)
end = datetime.datetime(2021,1,1)
data = downloadStockData(tickers,start,end)

yf.pdr_override()
data = pd.DataFrame()
for i in range(len(tickers)):
    try:
        o = pdr.get_data_yahoo(tickers[i],start, end)
        data[str(tickers[i])] = o["Close"]
    except:
        print('error')
        print(i)

[*********************100%%**********************]  48 of 48 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['HDFC.NS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2013-01-01 00:00:00 -> 2021-01-01 00:00:00)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['HDFC.NS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2013-01-01 00:00:00 -> 2021-01-01 00:00:00)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

In [11]:
data.dropna(inplace = True)
data

,ADANIPORTS.NS,TECHM.NS,BRITANNIA.NS,TCS.NS,SUNPHARMA.NS,INFY.NS,HINDALCO.NS,DRREDDY.NS,HCLTECH.NS,HINDUNILVR.NS,...,ONGC.NS,SBIN.NS,RELIANCE.NS,BAJFINANCE.NS,AXISBANK.NS,HDFCBANK.NS,TATAMOTORS.NS,ICICIBANK.NS,POWERGRID.NS,INDUSINDBK.NS
Date,,,,,,,,,,,,,,,,,,,,,


In [9]:
print(data.shape)

(0, 48)


In [8]:
def findCointegrationValue(data, p_value_threshold=0.05):
    """
    This function finds pairs of stocks with a p-value less than a given threshold.

    Params:
        data              : DataFrame containing closing prices for each stock on each trading day.
        p_value_threshold : The threshold value for the p-value of pairs of stocks.
                           Pairs are considered cointegrated if the p-value is less than this value.

    Returns:
        score   : Array of cointegration scores for each pair of stocks.
        pvalue  : Array of p-values for each pair of stocks.
        pairs   : List of pairs with p-values less than the threshold.
    """
    n = data.shape[1]
    score_ = np.zeros((n, n))
    pvalue_ = np.zeros((n, n))
    keys = data.columns
    pairs = []

    for i in range(n):
        for j in range(i + 1, n):
            s1 = data[keys[i]]
            s2 = data[keys[j]]
            result = coint(s1, s2)
            score_[i, j] = result[0]
            pvalue_[i, j] = result[1]

            if result[1] < p_value_threshold:
                pairs.append((keys[i], keys[j], result[1]))

    return score_, pvalue_, pairs


scores_final, pvalues_final, pairs_final =  findCointegrationValue(data=data)

def getBestKCointergratedStocks(data,k=10,p_value_threshold=0.05):
    """
        This functions find out the k pairs with the least p value and all of them should be less than the threshold
        Params:
            data               : The list of tuples containing X, Y, and corresponding pvalue (< p_value_threshold)
            k                  : the number of pairs to be found out
            p_value_threshold  : the threshold value for the p value of pairs of stocks. We consider a 2 stocks to be cointegrted only if the p value is less than this value

        Returns:
            the p value for all the pairs of stocks , the list k pairs with the least p value and their p values . If k such pairs doesn't exists return how many of them occur
    """

    df = pd.DataFrame(data, columns=['X', 'Y', 'pvalue'])

    return df.sort_values(by='pvalue').head(k)

df = getBestKCointergratedStocks(pairs_final, p_value_threshold=0.05)

ValueError: ignored

In [ ]:
df

In [ ]:
def check_for_stationarity(X, cutoff=0.01):
        # H_0 in adfuller is unit root exists (non-stationary)\n",
        # We must observe significant p-value to convince ourselves that the series is stationary\n",
        pvalue = adfuller(X)[1]
        if pvalue < cutoff:
            print ('p-value = ' + str(pvalue) + ' The series ' + X.name +' is likely stationary')
            return True
        else:
            print ('p-value = ' + str(pvalue) + ' The series ' + X.name +' is likely non-stationary')
            return False

In [ ]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

X1 = data["HDFC.NS"]
X2 = data["BAJAJFINSV.NS"]
X1 = sm.add_constant(X1)
results = sm.OLS(X2, X1).fit()

results.params

In [ ]:
(5.377*data["HDFC.NS"]-data["BAJAJFINSV.NS"]).plot()
adfuller(5.377*data["HDFC.NS"]-data["BAJAJFINSV.NS"])[1]

In [ ]:
tickers = ['HDFC.NS','BAJAJFINSV.NS']
start = datetime.datetime(2020,11,20)
end = datetime.datetime(2021,6,30)
#data = downloadStockData(tickers,start,end)

yf.pdr_override()
datanew = pd.DataFrame()
for i in range(len(tickers)):
    try:
        o = pdr.get_data_yahoo(tickers[i],start, end)
        datanew[str(tickers[i])] = o["Close"]
    except:
        print('error')
        print(i)

In [ ]:
datanew

In [ ]:
import numpy as np
import pandas as pd
import statsmodels
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import coint   # use this library to find this out the p_value
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import datetime

def trade8(S1, S2,window, MAX_SHORTING_AMOUNT, MAX_TRANSACTION_AT_A_TIME,error_bound1,error_bound2):

    spread = []
    beta = 5.3397
    spread.append(beta*S1 - S2)
    df = pd.DataFrame(spread)
    df = df.T
    df["HDFC.NS"] = datanew["HDFC.NS"]
    df["BAJAJFINSV.NS"] = datanew["BAJAJFINSV.NS"]
    df["Roll mean"] = df[0].rolling(window = 30).mean()
    df["Roll std"] = df[0].rolling(window = 30).std()
    df["zscore"] = (df[0]-df["Roll mean"])/df["Roll std"]
    df = df.drop(["Roll mean", "Roll std"], axis = 1)
    df = df.dropna()
    df = df.reset_index(drop=True)
    print(df)
    df["zscore"].plot()
    print(adfuller(df["zscore"].dropna())[1])
    df["Buy"] = 0
    df["Sell"] = 0
    for i in range(len(df)) :
      if i==0 :
        df.loc[i,"Buy"] = 1 if df.loc[i,"zscore"]>1.5 and df.loc[i,"zscore"]<2 else 0
      else :
        if df.loc[i,"zscore"]>1.5 and df.loc[i,"zscore"]<2:
          df.loc[i,"Buy"] = 1
        if df.loc[i-1, "Buy"] == 1 and df.loc[i,"zscore"]>1 and df.loc[i,"zscore"]<2:
          df.loc[i,"Buy"] = 1
    for i in range(len(df)) :
      if i==0 :
        df.loc[i,"Sell"] = 1 if df.loc[i,"zscore"]<-1.5 and df.loc[i,"zscore"]>-2 else 0
      else :
        if df.loc[i,"zscore"]<-1.5 and df.loc[i,"zscore"]>-2:
          df.loc[i,"Sell"] = 1
        if df.loc[i-1, "Sell"] == 1 and df.loc[i,"zscore"]<-1 and df.loc[i,"zscore"]>-2:
          df.loc[i,"Sell"] = 1
    money = 0
    for i in range(len(df)-1) :
      if df.loc[i,"Buy"] == 1 :
        money += beta*(df.loc[i+1,"HDFC.NS"]-df.loc[i,"HDFC.NS"])
        #print("Long profit HDFC %s" %(beta*(df.loc[i+1,"HDFC.NS"]-df.loc[i,"HDFC.NS"]))  )
        money -= (df.loc[i+1,"BAJAJFINSV.NS"] - df.loc[i,"BAJAJFINSV.NS"])
        #print("Short profit BAJAJFINSV %s" %(df.loc[i,"BAJAJFINSV.NS"] - df.loc[i+1,"BAJAJFINSV.NS"]))
        print("Hedged profit %s" %(df.loc[i,"BAJAJFINSV.NS"] - df.loc[i+1,"BAJAJFINSV.NS"] + beta*(df.loc[i+1,"HDFC.NS"]-df.loc[i,"HDFC.NS"])))

      elif df.loc[i,"Sell"] == 1:
        money += (df.loc[i+1,"BAJAJFINSV.NS"] - df.loc[i,"BAJAJFINSV.NS"])
        #print("Long profit BAJAJFINSV %s" %(df.loc[i+1,"BAJAJFINSV.NS"] - df.loc[i,"BAJAJFINSV.NS"]))
        money -= beta*(df.loc[i+1,"HDFC.NS"]-df.loc[i,"HDFC.NS"])
        #print("Short profit HDFC %s" %(beta*(df.loc[i,"HDFC.NS"]-df.loc[i+1,"HDFC.NS"])) )
        print("Hedged profit %s" %(beta*(df.loc[i,"HDFC.NS"]-df.loc[i+1,"HDFC.NS"]) + df.loc[i+1,"BAJAJFINSV.NS"] - df.loc[i,"BAJAJFINSV.NS"]))

    print(df)

##print(df.head(60))
    print(money)

trade8( datanew['HDFC.NS'],datanew['BAJAJFINSV.NS'], 60,10000,5,0.5,0.25)

# Reaons why those 2 stocks are similar. (Explain for any 2 stocks in this text cell)

We know that the Pairs trading strategies speculate on future convergence of spread between similar securities. Similarity concerns industry, sector, market capitalization, and other common exposures that might imply a comovement between stocks.

Take example of Adobe and Microsoft, These companies share the same sector, i.e., IT, and even their Industry is same, i.e., Software—Infrastructure. They deal with same type of technologies such as 3d modelling, Cloud systems, Application Software.

Hence it can be understood that companies sharing common industry and type of sector, are leveraged by the macroecomic factors like GDP growth, etc has a similar kind of effect on their trade. As we know pairs trading is a market neutral trading strategy enabling traders to profit from virtually any market conditions: uptrend, downtrend, or sideways movement and both these stocks fulfill this in as a pair. So, that's why the particular stocks are similar.